# Amazon Kendra를 이용한 "매장 현장 직원을 위한 Q&A 챗봇 

---

이 노트북에서는 매장 현장 직원을 위한 Q&A 챗봇을 구현하기 위한 생성형 AI 기반의 대화식 답변(Generative Conversational Answers) 기능을 위해 [Amazon Kendra](https://aws.amazon.com/kendra/)과 [Amazon Bedrock](https://aws.amazon.com/bedrock/)를 사용합니다.

완전관리형 지능형 검색서비스인 Amazon Kendra의 기능을 LLM과 결합하여 RAG 워크플로를 구현한다면 엔터프라이즈 콘텐츠에 대한 대화 경험을 제공하는 GenAI 애플리케이션을 쉽게 만들 수 있습니다. 

노트북은 아래와 같은 구성으로 이루어져있습니다.
1. RAG 사용 사례를 위한 문서 업로드
2. Amazon Kendra 검색엔진 사용을 위한 LangChain의 Amazon Kendra Retrieve API
3. 웹앱을 쉽게 만들 수 있는 오픈 소스 Python 라이브러리 Streamlit을 이용한 챗봇 서비스 구현 

본 노트북에서 사용되는 일반적인 리테일 회사에서 사용될 수 있는 가상의 문서를 참고했습니다.
- 고객 응대 및 서비스 원칙
- 근무 시간과 스케줄
- 매장 업무 메뉴얼 목차
- 상품 진열과 정리 원칙
- 일일 근무 시간 및 주간 근무 일정 규정
- 근태 규정
- 팀원과 상사와의 협력

Amazon Kendra와 Amazon Bedrock을 이용한 RAG 사례는 아래 다이어그램을 참조하세요.

<img src="images/rag-architecture.png" width="800"/>

---

## RAG 사례를 위한 Amazon S3 문서 업로드

In [2]:
# install Microsoft Word (.docx) python library
%pip install --quiet python-docx

Note: you may need to restart the kernel to use updated packages.


In [5]:
import glob
import os
import boto3

In [6]:
# RAG에 이용될 업로드 문서 확인
office_files = glob.glob("./data/*")
for filename in office_files:
    print(filename)

./data/product_display.docx
./data/attendance_policy.docx
./data/customer_service.docx
./data/working_schedule.docx
./data/store_manual_contents.docx
./data/team_collaboration.docx
./data/daily_weekly_schedule.docx


## 문서 확인

---

- 문서는 아래와 같은 내용으로 구성되어있습니다.

  예) 상품 진열과 정리 원칙 (product_display.docx) 파일 일부

<img src="images/docx_example.png" width="800"/>


In [7]:
import docx 

document = docx.Document('./data/product_display.docx')
for line, p in enumerate(document.paragraphs):
    print(p.text)

아래는 진열 및 정리 원칙에 대한 일반적인 내용입니다:
1. 상품 진열 계획:
상품 진열은 매장 레이아웃 및 계획에 따라 이루어져야 합니다. 매장 내 어떤 상품이 어디에 배치될지 계획이 필요합니다.
2. 제품 분류:
상품은 유형 또는 카테고리에 따라 분류되어야 합니다. 예를 들어, 식료품, 가정용품, 생활용품 등으로 구분될 수 있습니다.
3. 유통기한 확인:
상품 진열 시에는 유통기한을 확인하고 만기 상품은 배치하지 않아야 합니다.
4. 인기 상품 우선:
가장 인기 있는 상품은 눈에 잘 띄게 배치해야 합니다. 고객들이 쉽게 찾을 수 있도록 인기 상품을 전략적으로 배치하세요.
5. 선반의 높이:
다양한 선반 높이를 활용하여 고객이 편리하게 상품에 접근할 수 있도록 합니다. 무거운 상품은 허리 레벨에, 가벼운 상품은 높은 선반에 배치할 수 있습니다.
6. 청결과 정돈:
상품은 항상 청결하게 유지되어야 하며, 훼손된 제품은 정리되어야 합니다. 상품의 잔여량과 청결도를 주기적으로 확인하세요.
7. 가격 표시:
모든 상품에는 가격이 명확하게 표시되어야 합니다. 가격 표시가 눈에 잘 띄도록 하세요.
8. 브랜드 및 판매 프로모션 고려:
브랜드와 판매 프로모션에 따라 상품을 배치하세요. 고객이 할인 상품이나 특별 프로모션을 찾을 수 있도록 하세요.
9. 무료 공간 유지:
고객의 편의를 위해 통행로와 출입구 주변을 비우고, 화재 안전을 고려하여 비상구를 막지 마세요.
10. 주기적인 재고 회전:
상품은 FIFO (First-In, First-Out) 원칙에 따라 배치되어야 하며, 유통기한이 다가오거나 만료된 상품은 즉시 제거해야 합니다.
11. 고객의 의견 수용:
고객의 피드백을 듣고 상품 진열 및 정리를 개선하기 위해 활용하세요.
12. 안전 규정 준수:
상품을 진열할 때 안전 규정을 준수하며, 상품이 넘어지지 않도록 안전하게 고정하세요.
상품 진열 및 정리는 고객 경험과 매장 운영에 큰 영향을 미치는 중요한 부분이므로, 이러한 원칙을 준수하여 효과적으로 관리해야 합니다.

---

## 문서 S3 업로드

boto3 라이브러리를 이용해서 S3 버킷에 문서들을 모두 업로드힙니다.

In [9]:
STACK_NAME = 'genai-workshop'

cf_client = boto3.client('cloudformation')
response = cf_client.describe_stacks()

for output in response["Stacks"]:
    stackName = output["StackName"]
    if stackName.find('Kendra') > 0:
        response = cf_client.describe_stacks(StackName=stackName)
        for output in response["Stacks"][0]["Outputs"]:
            keyName = output["OutputKey"]
            if keyName == "S3Bucket":
                BUCKET_NAME = output["OutputValue"]
            if keyName == "KendraIndex":
                KENDRA_INDEX = output["OutputValue"]

print('S3 Bucket Name: ', BUCKET_NAME)
print('Kendra Index ID: ', KENDRA_INDEX)

S3 Bucket Name:  kendra-workshop-0b89bf00-8144-11ee-9a4c-0a688d7082d9
Kendra Index ID:  8a91c3ba-8a56-4804-9b82-13d830a538dc


In [18]:
s3 = boto3.client('s3')

for filename in office_files:
    key = os.path.basename(filename)
    print("Putting ", filename,key)
    s3.upload_file(filename, BUCKET_NAME, key)

Putting  ./data/product_display.docx product_display.docx
Putting  ./data/attendance_policy.docx attendance_policy.docx
Putting  ./data/customer_service.docx customer_service.docx
Putting  ./data/working_schedule.docx working_schedule.docx
Putting  ./data/store_manual_contents.docx store_manual_contents.docx
Putting  ./data/team_collaboration.docx team_collaboration.docx
Putting  ./data/daily_weekly_schedule.docx daily_weekly_schedule.docx


Note: S3에 파일을 업로드할 경우 Kendra Index 동기화 이벤트가 발생되어 자동으로 인덱싱이 진행됩니다.

---

## LangChain의 RetrievalQA Chain을 이용

LangChain이 Amazon Bedrock의 Claude-2 모델을 사용하도록 LLM 정의

In [12]:
%pip install --quiet langchain==0.0.309

Note: you may need to restart the kernel to use updated packages.


In [13]:
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"  # E.g. "us-east-1"
os.environ["BEDROCK_ENDPOINT_URL"] = "https://bedrock-runtime.us-west-2.amazonaws.com"  # E.g. "https://..."

In [14]:
from langchain.chains import RetrievalQA

os.environ["AWS_DEFAULT_REGION"] = os.environ["AWS_DEFAULT_REGION"]  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "bedrock_claude"
os.environ["BEDROCK_ENDPOINT_URL"] = os.environ["BEDROCK_ENDPOINT_URL"]  # E.g. "https://..."

session = boto3.Session(
    profile_name=os.environ.get("AWS_PROFILE")
) # sets the profile name to use for AWS credentials

bedrock = session.client(
    service_name='bedrock-runtime', # creates a Bedrock client
    region_name=os.environ.get("AWS_DEFAULT_REGION"),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL")
) 

from langchain.llms.bedrock import Bedrock

# - create the Anthropic Model
llm = Bedrock(model_id="anthropic.claude-v2", client=bedrock, model_kwargs={'max_tokens_to_sample':1000, 'temperature': 0})

---

## AmazonKendraRetriever 설정

LangChain이 Kendra 검색 결과를 가져오기 위해 AmazonKendraRetriever 이용

In [15]:
from langchain.retrievers import AmazonKendraRetriever

retriever = AmazonKendraRetriever(
    index_id=KENDRA_INDEX,
    region_name=os.environ.get("AWS_DEFAULT_REGION", None),
    top_k=3,
    attribute_filter = {
        "EqualsTo": {      
            "Key": "_language_code",
            "Value": {
                "StringValue": "ko"
            }
        }
    }
)

## Kendra Retriever로 문서 검색을 확인

In [19]:
# "상품 진열 방법"의 Kendra 검색 결과 확인
retriever.get_relevant_documents("상품 진열 방법")

[Document(page_content='Document Title: store_manual_contents\nDocument Excerpt: \n1. 소개 · 회사 소개 · 메뉴얼 목적 · 업무 환경 소개 2. 직무 소개 · 직무 개요 · 직무 명칭 및 설명 · 직무 유형 3. 업무 시간과 스케줄 · 근무 시간 및 스케줄 · 휴식 시간 · 휴가 신청 및 승인 절차 4. 업무 의무 · 기본 업무 의무 · 품질 기준 및 고객 서비스 · 안전 및 위생 규정 준수 5. 상품 배치 및 정리 · 상품 배치 방법 · 진열 및 정리 원칙 · 유통기한 및 상품 관리 6. 고객 서비스 · 고객 응대 및 서비스 원칙 · 고객 문의 처리 방법 · 불만사항 및 갈등 해결 7. 결제 및 레지스터 업무 · 레지스터 사용법 · 현금 및 카드 거래 처리 · 오류 처리 방법 8. 안전 및 위생 · 안전 규칙 및 절차 · 위생 규정 및 실천 · 비상 상황 대응 9. 팀 협력 · 팀 작업 및 소통 · 상사와의 협력 · 직원 간 협력 10. 교육 및 권리 · 교육 및 역량 개발 · 직원 권리 및 복리후생 혜택 11. 업무 메뉴얼 업데이트 · 메뉴얼 업데이트 절차 · 변경 사항 통보 12. 부록 · 관련 양식 및 절차서 · 응급 상황 연락처\n', metadata={'result_id': 'aa32bd1d-dee0-406b-babc-dde45d9309c1-47737b41-50e3-4ac2-8983-0ae6f90cd9b9', 'document_id': 's3://kendra-workshop-0b89bf00-8144-11ee-9a4c-0a688d7082d9/store_manual_contents.docx', 'source': 'https://s3.us-west-2.amazonaws.com/kendra-workshop-0b89bf00-8144-11ee-9a4c-0a688d7082d9/store_manual_contents.docx', 'title': 'store_manual_contents', 'ex

---

## Prompt Template 설정

질문와 답변 형태의 대화를 위한 Prompt를 Anthroipc Claude의 Prompt format에 맞도록 아래와 같이 정의

<li> 영어

Human: This is a friendly conversation between a human and an AI. 
The AI is talkative and provides specific details from its context but limits it to 240 tokens.
If the AI does not know the answer to a question, it truthfully says it 
does not know.

Assistant: OK, got it, I'll be a talkative truthful AI assistant.

Human: Here are a few documents in <documents> tags:
<documents>
{context}
</documents>
Based on the above documents, provide a detailed answer for, {question} 
Answer **"시스템에 관련된 정보가 없습니다."** if not present in the document. 

<li> 한글

Assistant:
    
Human: 인간과 AI의 친근한 대화입니다.
AI는 말이 많고 상황에 따른 구체적인 세부 정보를 제공하지만 토큰 수는 240개로 제한됩니다.
AI가 질문에 대한 답을 모르면 알지 못한다고 사실대로 말합니다.

Assistant: 알겠습니다. 저는 말이 많고 진실된 AI 어시스턴트가 되어 드리겠습니다.

Human: 다음은 <documents> 태그에 있는 몇 가지 문서입니다.
<문서>
{문맥}
</문서>
위 문서를 바탕으로 {question}에 대한 자세한 답변을 제공해 주세요.
문서에 관련된 내용이 없으면 "시스템에 관련된 정보는 없습니다."라고 답변하세요.

Assistant:

In [20]:
from langchain.prompts import PromptTemplate

prompt_template = """

Human: This is a friendly conversation between a human and an AI. 
The AI is talkative and provides specific details from its context but limits it to 240 tokens.
If the AI does not know the answer to a question, it truthfully says it 
does not know.

Assistant: OK, got it, I'll be a talkative truthful AI assistant.

Human: Here are a few documents in <documents> tags:
<documents>
{context}
</documents>
Based on the above documents, provide a detailed answer for, {question} 
Answer "시스템에 관련된 정보가 없습니다." if not present in the document. 

Assistant:"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [21]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # inserts all documents into a prompt and passes that prompt to an LLM
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)


In [22]:
# result = qa.run(question)
query = "상품 진열 방법 알려줘"
result = qa({"query": query})

In [23]:
# print(result.strip())
print(result['result'])
result['source_documents']

 상품 진열 방법에 대한 내용은 두번째 문서인 product_display에서 찾을 수 있습니다. 

product_display 문서에 따르면 상품 진열 방법의 주요 원칙은 다음과 같습니다:

1. 상품 진열 계획 수립 

2. 상품 카테고리별 분류

3. 유통기한 확인

4. 인기 상품 우선 진열

5. 다양한 높이의 선반 활용

6. 상품 청결 유지 및 정리

구체적으로, 상품 진열 시에는 먼저 매장 레이아웃과 계획에 따라 진열 계획을 세워야 합니다. 그리고 상품을 유형별로 분류하여 진열하며, 유통기한이 지난 상품은 제외해야 합니다. 인기 상품은 전략적으로 눈에 잘 띄는 곳에 배치하고, 다양한 높이의 선반을 사용하여 고객이 접근성을 높입니다. 또한 상품은 항상 청결하게 관리되어야 합니다.


[Document(page_content='Document Title: store_manual_contents\nDocument Excerpt: \n1. 소개 · 회사 소개 · 메뉴얼 목적 · 업무 환경 소개 2. 직무 소개 · 직무 개요 · 직무 명칭 및 설명 · 직무 유형 3. 업무 시간과 스케줄 · 근무 시간 및 스케줄 · 휴식 시간 · 휴가 신청 및 승인 절차 4. 업무 의무 · 기본 업무 의무 · 품질 기준 및 고객 서비스 · 안전 및 위생 규정 준수 5. 상품 배치 및 정리 · 상품 배치 방법 · 진열 및 정리 원칙 · 유통기한 및 상품 관리 6. 고객 서비스 · 고객 응대 및 서비스 원칙 · 고객 문의 처리 방법 · 불만사항 및 갈등 해결 7. 결제 및 레지스터 업무 · 레지스터 사용법 · 현금 및 카드 거래 처리 · 오류 처리 방법 8. 안전 및 위생 · 안전 규칙 및 절차 · 위생 규정 및 실천 · 비상 상황 대응 9. 팀 협력 · 팀 작업 및 소통 · 상사와의 협력 · 직원 간 협력 10. 교육 및 권리 · 교육 및 역량 개발 · 직원 권리 및 복리후생 혜택 11. 업무 메뉴얼 업데이트 · 메뉴얼 업데이트 절차 · 변경 사항 통보 12. 부록 · 관련 양식 및 절차서 · 응급 상황 연락처\n', metadata={'result_id': '8d05d725-c599-4566-b1dc-4b5c43868e02-2071c02e-cf35-40af-94a1-92b8f2333d9b', 'document_id': 's3://kendra-workshop-0b89bf00-8144-11ee-9a4c-0a688d7082d9/store_manual_contents.docx', 'source': 'https://s3.us-west-2.amazonaws.com/kendra-workshop-0b89bf00-8144-11ee-9a4c-0a688d7082d9/store_manual_contents.docx', 'title': 'store_manual_contents', 'ex

## 빠른 응답을 위해 claude-instant-v1 모델을 사용

In [24]:
llm = Bedrock(model_id="anthropic.claude-instant-v1", client=bedrock, model_kwargs={'max_tokens_to_sample':1000, 'temperature': 0})
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff", # inserts all documents into a prompt and passes that prompt to an LLM
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [25]:
%%time
query = "FIFO는 무슨 뜻이야?"
result = qa({"query": query})
print(result['result'])
result['source_documents']

 위 문서에서 FIFO(First-In, First-Out)는 상품을 배치할 때 가장 먼저 들어온 상품부터 순서대로 처리한다는 의미의 방식을 말합니다. 문서에서 "상품은 FIFO(First-In, First-Out) 원칙에 따라 배치되어야 하며, 유통기한이 다가오거나 만료된 상품은 즉시 제거해야 합니다."라고 설명하고 있습니다.
CPU times: user 11.4 ms, sys: 613 µs, total: 12.1 ms
Wall time: 3.12 s


[Document(page_content='Document Title: product_display\nDocument Excerpt: \n5. 선반의 높이: · 다양한 선반 높이를 활용하여 고객이 편리하게 상품에 접근할 수 있도록 합니다. 무거운 상품은 허리 레벨에, 가벼운 상품은 높은 선반에 배치할 수 있습니다. 6. 청결과 정돈: · 상품은 항상 청결하게 유지되어야 하며, 훼손된 제품은 정리되어야 합니다. 상품의 잔여량과 청결도를 주기적으로 확인하세요. 7. 가격 표시: · 모든 상품에는 가격이 명확하게 표시되어야 합니다. 가격 표시가 눈에 잘 띄도록 하세요. 8. 브랜드 및 판매 프로모션 고려: · 브랜드와 판매 프로모션에 따라 상품을 배치하세요. 고객이 할인 상품이나 특별 프로모션을 찾을 수 있도록 하세요. 7. 가격 표시: · 모든 상품에는 가격이 명확하게 표시되어야 합니다. 가격 표시가 눈에 잘 띄도록 하세요. 8. 브랜드 및 판매 프로모션 고려: · 브랜드와 판매 프로모션에 따라 상품을 배치하세요. 고객이 할인 상품이나 특별 프로모션을 찾을 수 있도록 하세요. 9. 무료 공간 유지: · 고객의 편의를 위해 통행로와 출입구 주변을 비우고, 화재 안전을 고려하여 비상구를 막지 마세요. 10. 주기적인 재고 회전: · 상품은 FIFO (First-In, First-Out) 원칙에 따라 배치되어야 하며, 유통기한이 다가오거나 만료된 상품은 즉시 제거해야 합니다. 11. 고객의 의견 수용: · 고객의 피드백을 듣고 상품 진열 및 정리를 개선하기 위해 활용하세요. 12. 안전 규정 준수: · 상품을 진열할 때 안전 규정을 준수하며, 상품이 넘어지지 않도록 안전하게 고정하세요. 상품 진열 및 정리는 고객 경험과 매장 운영에 큰 영향을 미치는 중요한 부분이므로, 이러한 원칙을 준수하여 효과적으로 관리해야 합니다.\n', metadata={'result_id': '744d51c0-6cd2-4c44-bb34-2f3f905dc7eb-fa26adcd-83bc-4902-

## 위의 코드를 kendra_claude.py로 저장

In [26]:
%store KENDRA_INDEX

Stored 'KENDRA_INDEX' (str)


In [27]:
print(KENDRA_INDEX)

8a91c3ba-8a56-4804-9b82-13d830a538dc


## LLM을 이용한 Q&A 챗봇 모듈을 kendra_claude.py로 저장

In [28]:
%%writefile kendra_claude.py

import sys
import os

import boto3
from langchain.retrievers import AmazonKendraRetriever
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms.bedrock import Bedrock

def build_chain():

  session = boto3.Session(
      profile_name=os.environ.get("AWS_PROFILE")
  ) 
  boto3_bedrock = session.client(
    service_name='bedrock-runtime', 
    region_name=os.environ.get("AWS_DEFAULT_REGION"),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL")
  ) 
    
  region = os.environ["AWS_REGION"]
  kendra_index_id = "<YOUR_KENDRA_INDEX>" # Example: 65702b79-XXXX-XXXX-XXXX-9702f17fb994

  llm = Bedrock(model_id="anthropic.claude-v2", client=boto3_bedrock, model_kwargs={'max_tokens_to_sample':1000})
  
  retriever = AmazonKendraRetriever(
    index_id=kendra_index_id,
    region_name=os.environ.get("AWS_DEFAULT_REGION", None),
    top_k=3,
    attribute_filter = {
        "EqualsTo": {      
            "Key": "_language_code",
            "Value": {
                "StringValue": "ko"
            }
        }
    }
  )
  # prompt_template = """Human: Use the following pieces of context to provide a concise answer to the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
  prompt_template = """Human: Use the following pieces of context to provide a concise answer to the question at the end. If the answer is not in the context, just say "시스템에 관련 내용을 찾을 수 없습니다.", don't try to make up an answer.

  {context}

  Question: {question}
  Assistant:"""

  PROMPT = PromptTemplate(
      template=prompt_template, input_variables=["context", "question"]
  )


  qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
  )

  return qa

def run_chain(chain, prompt: str):
  return chain({"query": prompt})

Writing kendra_claude.py


In [29]:
# Kendra index 업데이트
!sed -i 's/<YOUR_KENDRA_INDEX>/{KENDRA_INDEX}/g' kendra_claude.py

## Streamlit 애플리케이션 실행을 위한 app.py 파일을 생성

In [30]:
%%writefile app.py

import streamlit as st
import sys

import kendra_claude as claude

USER_ICON = "images/user-icon.png"
AI_ICON = "images/ai-icon.png"
MAX_HISTORY_LENGTH = 5

if 'llm_chain' not in st.session_state:
    st.session_state['llm_app'] = claude
    st.session_state['llm_chain'] = claude.build_chain()

if 'chat_history' not in st.session_state:
    st.session_state['chat_history'] = []
    
if "chats" not in st.session_state:
    st.session_state.chats = [
        {
            'id': 0,
            'question': '',
            'answer': ''
        }
    ]

if "questions" not in st.session_state:
    st.session_state.questions = []

if "answers" not in st.session_state:
    st.session_state.answers = []

if "input" not in st.session_state:
    st.session_state.input = ""


st.markdown("""
        <style>
               .block-container {
                    padding-top: 32px;
                    padding-bottom: 32px;
                    padding-left: 0;
                    padding-right: 0;
                }
                .element-container img {
                    background-color: #000000;
                }

                .main-header {
                    font-size: 24px;
                }
        </style>
        """, unsafe_allow_html=True)

def write_logo():
    col1, col2, col3 = st.columns([5, 1, 5])
    with col2:
        st.image(AI_ICON, use_column_width='always') 


def write_top_bar():
    col1, col2, col3 = st.columns([1,10,2])
    with col1:
        st.image(AI_ICON, use_column_width='always')
    with col2:
        header = f"Amazon Bedrock이 제공하는 AI 서비스!"
        st.write(f"<h3 class='main-header'>{header}</h3>", unsafe_allow_html=True)
    with col3:
        clear = st.button("Clear Chat")
    return clear

clear = write_top_bar()

if clear:
    st.session_state.questions = []
    st.session_state.answers = []
    st.session_state.input = ""
    st.session_state["chat_history"] = []

def handle_input():
    input = st.session_state.input
    question_with_id = {
        'question': input,
        'id': len(st.session_state.questions)
    }
    st.session_state.questions.append(question_with_id)

    chat_history = st.session_state["chat_history"]
    if len(chat_history) == MAX_HISTORY_LENGTH:
        chat_history = chat_history[:-1]

    llm_chain = st.session_state['llm_chain']
    chain = st.session_state['llm_app']
    result = chain.run_chain(llm_chain, input)
    answer = result['result']
    chat_history.append((input, answer))
    
    document_list = []
    if 'source_documents' in result:
        for d in result['source_documents']:
            if not (d.metadata['source'] in document_list):
                document_list.append((d.metadata['source']))

    st.session_state.answers.append({
        'answer': result,
        'sources': document_list,
        'id': len(st.session_state.questions)
    })
    st.session_state.input = ""

def write_user_message(md):
    col1, col2 = st.columns([1,12])
    
    with col1:
        st.image(USER_ICON, use_column_width='always')
    with col2:
        st.warning(md['question'])


def render_result(result):
    answer, sources = st.tabs(['Answer', 'Sources'])
    with answer:
        render_answer(result['answer'])
    with sources:
        if 'source_documents' in result:
            render_sources(result['source_documents'])
        else:
            render_sources([])

def render_answer(answer):
    col1, col2 = st.columns([1,12])
    with col1:
        st.image(AI_ICON, use_column_width='always')
    with col2:
        st.info(answer['result'])

def render_sources(sources):
    col1, col2 = st.columns([1,12])
    with col2:
        with st.expander("Sources"):
            for s in sources:
                st.write(s)

    
#Each answer will have context of the question asked in order to associate the provided feedback with the respective question
def write_chat_message(md, q):
    chat = st.container()
    with chat:
        render_answer(md['answer'])
        render_sources(md['sources'])
    
        
with st.container():
  for (q, a) in zip(st.session_state.questions, st.session_state.answers):
    write_user_message(q)
    write_chat_message(a, q)

st.markdown('---')
input = st.text_input("질문을 해주세요!", key="input", on_change=handle_input)

Writing app.py


In [31]:
%%writefile requirements.txt

boto3==1.28.64
streamlit==1.20.0
langchain

Writing requirements.txt


In [32]:
%%writefile setup.sh

pip install --no-cache-dir -r requirements.txt
sudo yum install -y iproute
sudo yum install -y jq
sudo yum install -y lsof

Writing setup.sh


In [33]:
%%writefile run.sh

#!/bin/sh
CURRENTDATE=`date +"%Y-%m-%d %T"`
RED='\033[0;31m'
CYAN='\033[1;36m'
GREEN='\033[1;32m'
NC='\033[0m'
S3_PATH=$1

# Run the Streamlit app and save the output to "temp.txt"
streamlit run app.py > temp.txt & 

# Read the text file using cat
echo "Getting the URL to view your Streamlit app in the browser"

# Extract the last four digits of the port number from the Network URL
sleep 5
PORT=$(grep "Network URL" temp.txt | awk -F':' '{print $NF}' | awk '{print $1}' | tail -c 5)
echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Port Number ${PORT}" 



# Get Studio domain information
DOMAIN_ID=$(jq .DomainId /opt/ml/metadata/resource-metadata.json || exit 1)
RESOURCE_NAME=$(jq .ResourceName /opt/ml/metadata/resource-metadata.json || exit 1)
RESOURCE_ARN=$(jq .ResourceArn /opt/ml/metadata/resource-metadata.json || exit 1)

# Remove quotes from string
DOMAIN_ID=`sed -e 's/^"//' -e 's/"$//' <<< "$DOMAIN_ID"`
RESOURCE_NAME=`sed -e 's/^"//' -e 's/"$//' <<< "$RESOURCE_NAME"`
RESOURCE_ARN=`sed -e 's/^"//' -e 's/"$//' <<< "$RESOURCE_ARN"`
RESOURCE_ARN_ARRAY=($(echo "$RESOURCE_ARN" | tr ':' '\n'))

# Get Studio domain region
REGION=$(echo "${RESOURCE_ARN_ARRAY[3]}")

# Check if it's Collaborative Space
SPACE_NAME=$(jq .SpaceName /opt/ml/metadata/resource-metadata.json || exit 1)

# if it's not a collaborative space 
if [ -z "$SPACE_NAME" ] || [ $SPACE_NAME == "null" ] ;
then
    # If it's a user-profile access
    echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Domain Id ${DOMAIN_ID}"
    STUDIO_URL="https://${DOMAIN_ID}.studio.${REGION}.sagemaker.aws"
    
# It is a collaborative space
else

    SEM=true
    SPACE_ID=

    # Check if Space Id was previously configured
    if [ -f /tmp/space-metadata.json ]; then
        SAVED_SPACE_ID=$(jq .SpaceId /tmp/space-metadata.json || exit 1)
        SAVED_SPACE_ID=`sed -e 's/^"//' -e 's/"$//' <<< "$SAVED_SPACE_ID"`

        if [ -z "$SAVED_SPACE_ID" ] || [ $SAVED_SPACE_ID == "null" ]; then
            ASK_INPUT=true
        else
            ASK_INPUT=false
        fi
    else
        ASK_INPUT=true
    fi

    # If Space Id is not available, ask for it
    while [[ $SPACE_ID = "" ]] ; do
        # If Space Id already configured, skeep the ask
        if [ "$ASK_INPUT" = true ]; then
            echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Please insert the Space Id from your url. e.g. https://${GREEN}<SPACE_ID>${NC}.studio.${REGION}.sagemaker.aws/jupyter/default/lab"
            read SPACE_ID
            SEM=true
        else
            SPACE_ID=$SAVED_SPACE_ID
        fi

        if ! [ -z "$SPACE_ID" ] && ! [ $SPACE_ID == "null" ] ;
        then
            while $SEM; do
                echo "${SPACE_ID}"
                read -p "Should this be used as Space Id? (y/N) " yn
                case $yn in
                    [Yy]* )
                        echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Domain Id ${DOMAIN_ID}"
                        echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Space Id ${SPACE_ID}"

                        jq -n --arg space_id $SPACE_ID '{"SpaceId":$space_id}' > /tmp/space-metadata.json

                        STUDIO_URL="https://${SPACE_ID}.studio.${REGION}.sagemaker.aws"

                        SEM=false
                        ;;
                    [Nn]* ) 
                        SPACE_ID=
                        ASK_INPUT=true
                        SEM=false
                        ;;
                    * ) echo "Please answer yes or no.";;
                esac
            done
        fi
    done
fi

echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Studio Url ${STUDIO_URL}"


link="${STUDIO_URL}/jupyter/${RESOURCE_NAME}/proxy/${PORT}/"

echo -e "${CYAN}${CURRENTDATE}: [INFO]:${NC} Starting Streamlit App"
echo -e "${CYAN}${CURRENTDATE}: [INFO]: ${GREEN}${link}${NC}"

exit 0
fi

Writing run.sh


# Streamlit 애플리케이션 실행

- 아래와 같이 Streamlit을 위한 Python 라이브러리 설치 및 실행 스크립트를 실행합니다.
<img src="images/streamlit-env.png" width="600"/>

```bash
cd aws-genai-for-retail/3_lab/
```
```bash
. setup.sh
```

- 아래와 같이 Streamlit 애플리케이션 실행을 위한 스크립트를 실행합니다.
<img src="images/streamlit-exe.png" width="1000"/>

```bash
. run.sh
```

- 위 스크립트 실행 결과 중 마지막 Streamlit 실행 링크를 클릭하면 새로운 브라우저 탭에 아래와 같이 QA 서비스에 접속할 수 있습니다.
<img src="images/streamlit-init.png" width="1000"/>

In [ ]:
- 위 스크립트 실행 결과 중 마지막 Streamlit 실행 링크를 클릭하면 새로운 브라우저 탭에 아래와 같이 QA 서비스에 접속할 수 있습니다.
<img src="images/streamlit-chat.png" width="1000"/>